In [16]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import datetime
import random
import seaborn as sns
import plotly.express as px

from collections import defaultdict
from scipy.stats import pearsonr

from sklearn.metrics import silhouette_score

In [17]:
df = pd.read_csv('df_scaled.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197914 entries, 0 to 197913
Data columns (total 41 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   povertyPercentage                 197914 non-null  float64
 1   males_percentage_per_city         197914 non-null  float64
 2   females_percentage_per_city       197914 non-null  float64
 3   killed_percentage_per_district    197914 non-null  float64
 4   injured_percentage_per_district   197914 non-null  float64
 5   arrested_percentage_per_district  197914 non-null  float64
 6   unharmed_percentage_per_district  197914 non-null  float64
 7   killed_percentage                 197914 non-null  float64
 8   injury_percentage                 197914 non-null  float64
 9   arrest_percentage                 197914 non-null  float64
 10  unharmed_rate                     197914 non-null  float64
 11  winning_party_percentage          197914 non-null  f

In [18]:
df = df.loc[df['state'] == 'illinois']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15935 entries, 1729 to 197852
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   povertyPercentage                 15935 non-null  float64
 1   males_percentage_per_city         15935 non-null  float64
 2   females_percentage_per_city       15935 non-null  float64
 3   killed_percentage_per_district    15935 non-null  float64
 4   injured_percentage_per_district   15935 non-null  float64
 5   arrested_percentage_per_district  15935 non-null  float64
 6   unharmed_percentage_per_district  15935 non-null  float64
 7   killed_percentage                 15935 non-null  float64
 8   injury_percentage                 15935 non-null  float64
 9   arrest_percentage                 15935 non-null  float64
 10  unharmed_rate                     15935 non-null  float64
 11  winning_party_percentage          15935 non-null  float64
 12  kille

In [19]:
column_to_drop = ['n_participants','n_participants_child', 'n_participants_teen',
       'n_participants_adult', 'n_males', 'n_females', 'n_killed', 'n_injured',
       'n_arrested', 'n_unharmed','date', 'latitude', 'longitude','congressional_district', 'candidatevotes', 'totalvotes', 'total_votes_for_state', 'involve_killing', 'pc1', 'pc2']
for col in df.columns:
    if df[col].dtype == 'object':
       column_to_drop.append(col)
column_to_drop

['n_participants',
 'n_participants_child',
 'n_participants_teen',
 'n_participants_adult',
 'n_males',
 'n_females',
 'n_killed',
 'n_injured',
 'n_arrested',
 'n_unharmed',
 'date',
 'latitude',
 'longitude',
 'congressional_district',
 'candidatevotes',
 'totalvotes',
 'total_votes_for_state',
 'involve_killing',
 'pc1',
 'pc2',
 'date',
 'state',
 'city_or_county',
 'incident_characteristics1',
 'party']

In [20]:
df_categorical = df[column_to_drop]
df_numeric = df.drop(columns=column_to_drop)

In [21]:
df_categorical.info()
df_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15935 entries, 1729 to 197852
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   n_participants             15935 non-null  float64
 1   n_participants_child       15935 non-null  int64  
 2   n_participants_teen        15935 non-null  int64  
 3   n_participants_adult       15935 non-null  int64  
 4   n_males                    15935 non-null  float64
 5   n_females                  15935 non-null  float64
 6   n_killed                   15935 non-null  float64
 7   n_injured                  15935 non-null  float64
 8   n_arrested                 15935 non-null  float64
 9   n_unharmed                 15935 non-null  float64
 10  date                       15935 non-null  object 
 11  latitude                   15935 non-null  float64
 12  longitude                  15935 non-null  float64
 13  congressional_district     15935 non-null  int6

# Hierarchical clustering (Agglomerative)

In [22]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from sklearn import metrics

In [25]:
table = []
for metric in ['cityblock','correlation', 'cosine', 'dice', 'euclidean']:
    methods = ['complete','single','average','weighted']
    if metric == 'euclidean':
        methods += ['ward','median','centroid']
    for method in methods:
        for truncation_mode in [None, 'lastp','level']:
            for level in [2,5,10,15]:
                for k in [6, 7,8,9,10]:
                    distances = pdist(df_numeric, metric=metric)
                    dlinkage = linkage(distances, metric=metric, method=method)
                    labels = fcluster(dlinkage,k,criterion='maxclust')
                    dendrogram(dlinkage, truncate_mode=truncation_mode, p=level, leaf_rotation=90)
                    plt.show()
                    table.append({'metric':metric, 'method':method, 'truncation_mode': truncation_mode, 'level':level, 'k':k, 'silhouette': silhouette_score(df_numeric, labels), 'DBscore':metrics.davies_bouldin_score(df_numeric, labels)})

pd.DataFrame(table)

In [ ]:

""" scatterplot(df[numeric_attributes+['pc1','pc2']], 'pc1','pc2',c_labels=labels,filename=os.path.join('hierarchical',directory, 'scatterplot.png'))

plots(df, labels, os.path.join('hierarchical',directory), attributes=numeric_attributes) """
